In [11]:
!pip3 install pandas scikit-learn numpy matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 1.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
import pandas as pd

# Update the path to your local NSL-KDD dataset file
train_data_url = '../NSL-KDD/KDDTrain+.txt'
test_data_url = '../NSL-KDD/KDDTest+.txt'

# Define column names
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
           'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
           'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
           'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
           'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack_type', 'difficulty']

# Load the data
train_data = pd.read_csv(train_data_url, names=columns)
test_data = pd.read_csv(test_data_url, names=columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Combine the train and test datasets for preprocessing
combined_data = pd.concat([train_data, test_data])

# Encode the categorical features
categorical_features = ['protocol_type', 'service', 'flag', 'attack_type']
for feature in categorical_features:
    encoder = LabelEncoder()
    combined_data[feature] = encoder.fit_transform(combined_data[feature])

# Split the combined dataset back into train and test datasets
train_data = combined_data.iloc[:len(train_data), :]
test_data = combined_data.iloc[len(train_data):, :]

In [ ]:
from sklearn.model_selection import train_test_split

X_train = train_data.drop(columns=['attack_type', 'difficulty'])
y_train = train_data['attack_type']
X_test = test_data.drop(columns=['attack_type', 'difficulty'])
y_test = test_data['attack_type']

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Create an Adaptive Boosting classifier
ab_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=0.5)

# Train the classifier
ab_clf.fit(X_train, y_train)

# Test the classifier
ab_y_pred = ab_clf.predict(X_test)

# Evaluate the classifier
print("Adaptive Boosting Classifier:")
print(classification_report(y_test, ab_y_pred))

In [ ]:
import numpy as np


class GreyWolfOptimizer:
    def __init__(self, obj_function, search_space, pack_size, n_iter):
        self.obj_function = obj_function
        self.search_space = search_space
        self.pack_size = pack_size
        self.n_iter = n_iter
        self.dim = len(search_space)

    def optimize(self):
        # Initialize pack positions
        pack_positions = np.array(
            [np.random.uniform(space[0], space[1], self.pack_size) for space in self.search_space]).T

        # Calculate objective function values for each position
        obj_values = np.array([self.obj_function(pos) for pos in pack_positions])

        # Find the initial alpha, beta, and delta wolves
        alpha_idx, beta_idx, delta_idx = np.argsort(obj_values)[:3]

        alpha = pack_positions[alpha_idx]
        beta = pack_positions[beta_idx]
        delta = pack_positions[delta_idx]

        # Start GWO iterations
        for iter_idx in range(self.n_iter):
            a = 2 - iter_idx * (2 / self.n_iter)  # Linearly decreasing 'a' parameter

          for iter_idx in range(self.n_iter):
            a = 2 - iter_idx * (2 / self.n_iter)  # Linearly decreasing 'a' parameter

            for i in range(self.pack_size):
                # Update positions of the pack members
                for j in range(self.dim):
                    r1, r2 = np.random.rand(2)  # Random coefficients

                    # Calculate the updating components for each of the three leading wolves
                    A_alpha = 2 * a * r1 - a
                    A_beta = 2 * a * r1 - a
                    A_delta = 2 * a * r1 - a

                    C_alpha = 2 * r2
                    C_beta = 2 * r2
                    C_delta = 2 * r2

                    D_alpha = abs(C_alpha * alpha[j] - pack_positions[i, j])
                    D_beta = abs(C_beta * beta[j] - pack_positions[i, j])
                    D_delta = abs(C_delta * delta[j] - pack_positions[i, j])

                    # Update the position of the current pack member
                    pack_positions[i, j] = (alpha[j] - A_alpha * D_alpha + beta[j] - A_beta * D_beta +
                                            delta[j] - A_delta * D_delta) / 3

                    # Clip the updated position to the search space bounds
                    pack_positions[i, j] = np.clip(pack_positions[i, j], self.search_space[j][0],
                                                   self.search_space[j][1])
                # Calculate the updated objective function value
                updated_obj_value = self.obj_function(pack_positions[i])

                # Update the alpha, beta, and delta wolves if necessary
                if updated_obj_value < obj_values[alpha_idx]:
                    delta_idx = beta_idx
                    beta_idx = alpha_idx
                    alpha_idx = i
                elif updated_obj_value < obj_values[beta_idx]:
                    delta_idx = beta_idx
                    beta_idx = i
                elif updated_obj_value < obj_values[delta_idx]:
                    delta_idx = i

                # Update the objective function value for the current pack member
                obj_values[i] = updated_obj_value

                # Update the positions of the alpha, beta, and delta wolves
                alpha = pack_positions[alpha_idx]
                beta = pack_positions[beta_idx]
                delta = pack_positions[delta_idx]


        # Return the best solution found
        return alpha

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Define objective function for GWO
def objective_function(params):
    max_depth, min_samples_split = int(params[0]), int(params[1])

    classifier = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # Use F1 score as the optimization metric
    return -f1_score(y_test, y_pred, average='micro')

# Define search space
search_space = [
    (1, 50),  # max_depth
    (2, 100)  # min_samples_split
]

# Create GWO optimizer
gwo_optimizer = GreyWolfOptimizer(obj_function=objective_function, search_space=search_space, pack_size=5, n_iter=50)

# Run GWO optimization
best_params = gwo_optimizer.optimize()

# Create the final classifier with optimized hyperparameters
gwo_clf = DecisionTreeClassifier(max_depth=int(best_params[0]), min_samples_split=int(best_params[1]))
gwo_clf.fit(X_train, y_train)
gwo_y_pred = gwo_clf.predict(X_test)

# Evaluate the classifier
print("Grey Wolf Optimizer Classifier:")
print(classification_report(y_test, gwo_y_pred))

In [ ]:
from sklearn.metrics import f1_score

ab_f1 = f1_score(y_test, ab_y_pred, average='micro')
gwo_f1 = f1_score(y_test, gwo_y_pred, average='micro')

print(f"Adaptive Boosting Classifier F1 Score: {ab_f1:.4f}")
print(f"Grey Wolf Optimizer Classifier F1 Score: {gwo_f1:.4f}")

if ab_f1 > gwo_f1:
    print("The best algorithm is: Adaptive Boosting Classifier")
else:
    print("The best algorithm is: Grey Wolf Optimizer Classifier")

In [ ]:
import pandas as pd

# Create a function to extract the required metrics
def extract_metrics(y_true, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)

    return {'Precision': precision, 'Recall': recall, 'F1 Score': f1, 'Accuracy': accuracy}

# Extract the metrics for both classifiers
ab_metrics = extract_metrics(y_test, ab_y_pred)
gwo_metrics = extract_metrics(y_test, gwo_y_pred)

# Create a DataFrame to display the results
results_df = pd.DataFrame({'Adaptive Boosting': ab_metrics, 'Grey Wolf Optimizer': gwo_metrics})

# Display the results in tabular format
results_df